In [1]:
import src.kgbench as kg
from src.rgcn_explainer_utils import *
import torch



/usr/local/Caskroom/miniconda/base/envs/RGCNExplainer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

# Get the current directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

# Check if the current directory is already the parent directory
if current_dir != '/Users/macoftraopia/Documents/GitHub/RGCN-Explainer':
    # Set the parent directory as the current directory
    os.chdir(parent_dir)
!pwd

/Users/macoftraopia/Documents/GitHub/RGCN-Explainer


In [11]:

from src.rgcn_explainer_utils import *
name = 'mdgender'
if name in ['aifb', 'mutag', 'bgs', 'am', 'mdgenre', 'amplus', 'mdgender']:
    data = kg.load(name, torch=True, final=False)
if 'IMDb' in name:    
    data = torch.load(f'data/IMDB/finals/{name}.pt')
if 'dbo' in name:
    data = torch.load(f'data/DBO/finals/{name}.pt')
get_relations(data)
relations = [data.i2rel[i][0] for i in range(len(data.i2rel))]
data.entities = np.append(data.triples[:,0].tolist(),(data.triples[:,2].tolist()))
data.triples = torch.Tensor(data.triples).to(int)
data.withheld = torch.Tensor(data.withheld).to(int)
data.training = torch.Tensor(data.training).to(int)
print('rel:', data.num_relations, 'ent:', data.num_entities, 'triples:', data.triples.shape)
print('training', data.training.shape, 'withheld', data.withheld.shape)
# data.entities = np.append(data.triples[:,0].detach().numpy(),(data.triples[:,2].detach().numpy()))
# relations = get_relations(data)

loaded data mdgender (75.42s).
rel: 154 ent: 349347 triples: torch.Size([1203789, 3])
training torch.Size([45321, 2]) withheld torch.Size([1000, 2])


In [14]:
# val = data.withheld[:,0]
# most_common_node = {}
# for i in val:
#     sub_edges, neighborhoods, sub_edges_tensor = find_n_hop_neighbors(data, 2,int(i), adj=True)

#     counter = Counter([num for tup in sub_edges for num in tup]).most_common(1)[0][0]
#     most_common_node[int(i)] = counter

In [4]:
if name=='dbo_gender':
    male = data.training[data.training[:,1] == 0][:,0].tolist()
    female = data.training[data.training[:,1] == 1][:,0].tolist()
    male_triples = [data.triples[i][1].tolist() for i in data.triples[:,1].tolist() if male[i] in  data.triples[:,0].tolist() ]
    female_triples = [data.triples[i][1].tolist() for i in data.triples[:,1].tolist() if female[i] in  data.triples[:,0].tolist() ]
    female_triples = [data.triples[i][1].tolist() for i in data.triples[:,1].tolist() if data.triples[i][0] in female ]
    def frequency_relations(data, subset=None, all = True):
        if all:
            freq = Counter(data.triples[:,1].tolist())
            print(freq)
        else:
            freq = Counter(subset)
            print(freq)
        sorted_freq = {data.i2r[k]: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)}
        return sorted_freq
    #f = frequency_relations(data,male, all = False)
    f = frequency_relations(data,female_triples, all = False)
    print(f)

In [12]:
import numpy as np
from collections import defaultdict

def select_matching_triples(triples_tensor, nodes_to_check):
    selected_triples = []

    # Iterate over the triples and check if any node matches with nodes_to_check
    for triple in triples_tensor:
        head, relation, tail = triple.tolist()
        if head in nodes_to_check or tail in nodes_to_check:
            selected_triples.append(triple)

    return torch.stack(selected_triples)

def compute_average_node_degree(triples_tensor):
    # Create a dictionary to store the node degrees
    node_degrees = {}

    # Count the node degrees using NumPy
    unique_nodes, counts = np.unique(triples_tensor[:, [0, 2]], return_counts=True)
    node_degrees.update(dict(zip(unique_nodes.tolist(), counts)))

    # Compute the average node degree and standard deviation using NumPy
    num_nodes = len(node_degrees)
    total_degree = sum(node_degrees.values())
    average_degree = total_degree / num_nodes

    degree_values = list(node_degrees.values())
    degree_std = np.std(degree_values)

    return average_degree, degree_std


def compute_average_degree_at_2_hops(triples_tensor):
    # Create an adjacency dictionary to store the neighbors of each node
    adjacency_dict = {}

    # Iterate over the triples and build the adjacency dictionary
    for head, _, tail in triples_tensor:
        if head not in adjacency_dict:
            adjacency_dict[head] = set()
        adjacency_dict[head].add(tail)

        if tail not in adjacency_dict:
            adjacency_dict[tail] = set()
        adjacency_dict[tail].add(head)

    # Compute the average degree at 2 hops for each node
    average_degrees_2_hops = []
    for node, neighbors in adjacency_dict.items():
        two_hop_neighbors = set()
        for neighbor in neighbors:
            two_hop_neighbors.update(adjacency_dict.get(neighbor, set()))
        average_degrees_2_hops.append(len(neighbors) + len(two_hop_neighbors))

    # Compute the average degree at 2 hops and its standard deviation using NumPy
    average_degrees_2_hops = np.array(average_degrees_2_hops)
    average_degree_2_hops = np.mean(average_degrees_2_hops)
    degree_std_2_hops = np.std(average_degrees_2_hops)

    return average_degree_2_hops, degree_std_2_hops


def compute_num_edges_to_node(triples_list, node):
    # Create an adjacency dictionary to store the neighbors of each node
    adjacency_dict = defaultdict(set)

    # Iterate over the triples and build the adjacency dictionary
    for head, _, tail in triples_list:
        adjacency_dict[head].add(tail)
        adjacency_dict[tail].add(head)

    # Compute the number of edges to the given node at 0 and 2 hops
    neighbors_0_hop = adjacency_dict.get(node, set())
    neighbors_2_hops = set()

    # Compute neighbors at 2 hops away
    for neighbor in neighbors_0_hop:
        neighbors_2_hops.update(adjacency_dict.get(neighbor, set()))

    num_edges_to_node_0_hop = len(neighbors_0_hop)
    num_edges_to_node_2_hops = len(neighbors_2_hops) - 1  # Exclude the starting node itself from 2-hop neighbors

    return num_edges_to_node_0_hop, num_edges_to_node_2_hops

In [16]:
import random 
from src.rgcn_explainer_utils import d_classes
num_samples_per_class = 4
random.seed(42)
dict_classes = d_classes(data)
sampled_data = []
for key in dict_classes:
    if key == 7 or key == 8:

        sampled_data.extend(random.sample(dict_classes[key], num_samples_per_class))
print(sampled_data)

[198665, 108380, 164115, 241860, 184391, 256831, 233709, 206749]


In [17]:
val = data.withheld[:, 0]

#selected_triples = select_matching_triples(data.triples, sampled_data)
sum_0, sum_2 = [], []
for i in sampled_data:
    num_edges_0_hop, num_edges_2_hops = compute_num_edges_to_node(data.triples.tolist(), int(i))
    sum_0.append(num_edges_0_hop)
    sum_2.append(num_edges_2_hops)

print('avg num edges validation 0 hop', np.round(np.mean(sum_0), 3), '±', np.round(np.std(sum_0), 3))
print('avg num edges validation 2 hop', np.round(np.mean(sum_2), 3), '±', np.round(np.std(sum_2), 3))
average_degree, std_degree = compute_average_node_degree(data.triples)
print("Average Node Degree:", np.round(average_degree,3), "±", np.round(std_degree,3))
average_degree_2_hops, degree_std_2_hops = compute_average_degree_at_2_hops(data.triples)
print("Average full Degree at 2 Hops:", np.round(average_degree_2_hops,3),'±', np.round(degree_std_2_hops,3))

avg num edges validation 0 hop 16.75 ± 8.089
avg num edges validation 2 hop 60618.375 ± 2387.198
Average Node Degree: 6.892 ± 138.437
Average full Degree at 2 Hops: 2.0 ± 0.0


In [ ]:
def object_type(v,h,data, relation_id = None,type=True):
    ''' Get the object class of a specific relation'''
    if type:
        relation_id = [i for i in range(data.num_relations) if 'type' in data.i2r[i]][-1]
    output_indices_v, output_values, value_indices = select_relation(v, data.num_entities, relation_id)
    output_indices_h, output_values, value_indices = select_relation(h, data.num_entities, relation_id)
    objects_types = match_to_triples(output_indices_v, output_indices_h,data, sparse=False)
    list = []
    for i in objects_types:
        list.append(data.i2e[i[2]][0])#.split('#')[1])
    result = Counter(list)
    return result

for node_idx in data.withheld[:,0]:
    h, v = torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_ver{node_idx}'), torch.load(f'chk/aifb_chk/hops_2_size_5e-05_lr_0.1_ent_-1_type_1_threshold_0.5_init_const_exp_/masked_adj/masked_hor{node_idx}')
    print(f'node {node_idx}:', object_type(v,h,data, 39))